In [1]:

import torch
import torch.nn as nn
from torch.nn import functional as F

# Hyperparameters
batch_size = 32  # how many independent sequences will we process in parallel?
block_size = 128  # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 500
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 384
n_head = 6
n_layer = 6
dropout = 0.2

torch.manual_seed(1337)
# We always start with a dataset to train on. Let's download the tiny shakespeare dataset
#wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

chars = sorted(list(set(text)))
vocab_size = len(chars)
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i + block_size] for i in ix])
    y = torch.stack([data[i + 1:i + block_size + 1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)
        q = self.query(x)
        wei = q @ k.transpose(-2, -1) * k.shape[-1] ** -0.5
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        v = self.value(x)
        out = wei @ v
        return out

class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

class GPTLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        B, T = idx.shape
        tok_emb = self.token_embedding_table(idx)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            targets = targets.view(B * T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]
            logits, loss = self(idx_cond)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx

model = GPTLanguageModel()
m = model.to(device)
print(sum(p.numel() for p in m.parameters()) / 1e6, 'M parameters')

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    xb, yb = get_batch('train')
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))

class SimpleTokenizer:
    def __init__(self, stoi, itos):
        self.stoi = stoi
        self.itos = itos

    def encode(self, text):
        return torch.tensor([self.stoi.get(c, self.stoi[' ']) for c in text], dtype=torch.long).unsqueeze(0)

    def decode(self, tokens):
        return ''.join(self.itos[token] for token in tokens if token in self.itos)

# Example character set mappings, replace these with your actual mappings
with open("input.txt", "r") as f:
    text = f.read()
chars = sorted(list(set(text)))  # Define 'text' variable as your dataset text content
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}

tokenizer = SimpleTokenizer(stoi, itos)

def generate_text(prompt, model, max_new_tokens=100):
    model.eval()
    with torch.no_grad():
        tokens = tokenizer.encode(prompt).to(device)
        generated_tokens = model.generate(tokens, max_new_tokens=max_new_tokens)
        generated_text = tokenizer.decode(generated_tokens[0].tolist())
    return generated_text

# Example of generating text for different types of prompts
def run_text_generation_tests(model):
    test_prompts = {
        "Coherence Test": "Once upon a time, there was a brave knight named",
        "Contextual Relevance Test": "Romeo stood beneath Juliet's balcony and whispered,",
        "Stylistic Alignment Test": "To be or not to be, that is",
        "Complexity Test": "In the darkest depths of the enchanted forest, the",
        "Cultural Reference Test": "Hark! What light through yonder window"
    }

    for test_name, prompt in test_prompts.items():
        print(f"{test_name} - Generated text:", generate_text(prompt, model, max_new_tokens=100))

# Assuming 'model' is your trained GPTLanguageModel instance
run_text_generation_tests(model)

10.739777 M parameters


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


step 0: train loss 4.3213, val loss 4.3130
step 500: train loss 1.9163, val loss 2.0195
step 1000: train loss 1.6001, val loss 1.7811
step 1500: train loss 1.4684, val loss 1.6593
step 2000: train loss 1.3887, val loss 1.5967
step 2500: train loss 1.3467, val loss 1.5801
step 3000: train loss 1.3074, val loss 1.5409
step 3500: train loss 1.2761, val loss 1.5302
step 4000: train loss 1.2418, val loss 1.5189
step 4500: train loss 1.2208, val loss 1.5124
step 4999: train loss 1.2033, val loss 1.5146

Most Lord Warwick, wiltune child him to him
Thought her sweet bound himself at to a MostleRy.
All is he
That thus turn thus, the well occk as; fear whose horse,
seen gave the time at Plain.
'Tis utchiary service, as sto it intends
Can enduuing his bloody; father doth each ended,
The revery with in noble set men twelves by his:
And he now the welvily will not bear a thoot;
Where he, and make concudy, what is Rome to run the know?
If have found one enemy and go a determity.

JULIET:
This is ton

In [6]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# Hyperparameters
batch_size = 32
block_size = 128
max_epochs = 5000
eval_interval = 500
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iterations = 200
embedding_dim = 384
num_heads = 6
num_layers = 6
dropout_rate = 0.2

torch.manual_seed(1337)

# Download and load the dataset
#dataset_url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
#!wget {dataset_url}
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# Preprocess the dataset
chars = sorted(list(set(text)))
vocab_size = len(chars)
char_to_index = {ch: i for i, ch in enumerate(chars)}
index_to_char = {i: ch for i, ch in enumerate(chars)}
encode = lambda s: [char_to_index[c] for c in s]
decode = lambda l: ''.join([index_to_char[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
train_data_len = int(0.9 * len(data))
train_data = data[:train_data_len]
val_data = data[train_data_len:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    indices = torch.randint(len(data) - block_size, (batch_size,))
    input_sequences = torch.stack([data[i:i + block_size] for i in indices])
    target_sequences = torch.stack([data[i + 1:i + block_size + 1] for i in indices])
    input_sequences, target_sequences = input_sequences.to(device), target_sequences.to(device)
    return input_sequences, target_sequences

@torch.no_grad()
def evaluate_model():
    model.eval()
    train_loss = torch.zeros(eval_iterations)
    val_loss = torch.zeros(eval_iterations)
    for k in range(eval_iterations):
        X, Y = get_batch('train')
        logits, loss = model(X, Y)
        train_loss[k] = loss.item()
        X, Y = get_batch('val')
        logits, loss = model(X, Y)
        val_loss[k] = loss.item()
    model.train()
    return train_loss.mean(), val_loss.mean()

class MultiHeadAttention(nn.Module):
    def __init__(self, embedding_dim, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.embedding_dim = embedding_dim
        self.num_heads = num_heads
        self.head_dim = embedding_dim // num_heads

        self.query_proj = nn.Linear(embedding_dim, embedding_dim)
        self.key_proj = nn.Linear(embedding_dim, embedding_dim)
        self.value_proj = nn.Linear(embedding_dim, embedding_dim)
        self.out_proj = nn.Linear(embedding_dim, embedding_dim)

        self.dropout = nn.Dropout(dropout_rate)
        self.attention_mask = torch.tril(torch.ones(block_size, block_size)).to(device)

    def forward(self, x):
        batch_size = x.size(0)

        query = self.query_proj(x).view(batch_size, -1, self.num_heads, self.head_dim).transpose(1, 2)
        key = self.key_proj(x).view(batch_size, -1, self.num_heads, self.head_dim).transpose(1, 2)
        value = self.value_proj(x).view(batch_size, -1, self.num_heads, self.head_dim).transpose(1, 2)

        attention_scores = torch.matmul(query, key.transpose(-2, -1)) / self.head_dim ** 0.5
        attention_scores = attention_scores.masked_fill(self.attention_mask[:x.size(1), :x.size(1)] == 0, float('-inf'))
        attention_probs = self.dropout(F.softmax(attention_scores, dim=-1))

        out = torch.matmul(attention_probs, value).transpose(1, 2).contiguous().view(batch_size, -1, self.embedding_dim)
        out = self.out_proj(out)

        return out

class FeedForwardLayer(nn.Module):
    def __init__(self, embedding_dim):
        super(FeedForwardLayer, self).__init__()
        self.fc1 = nn.Linear(embedding_dim, 4 * embedding_dim)
        self.fc2 = nn.Linear(4 * embedding_dim, embedding_dim)
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, x):
        out = self.fc1(x)
        out = F.relu(out)
        out = self.fc2(out)
        out = self.dropout(out)
        return out

class TransformerBlock(nn.Module):
    def __init__(self, embedding_dim, num_heads):
        super(TransformerBlock, self).__init__()
        self.attention = MultiHeadAttention(embedding_dim, num_heads)
        self.feed_forward = FeedForwardLayer(embedding_dim)
        self.layer_norm1 = nn.LayerNorm(embedding_dim)
        self.layer_norm2 = nn.LayerNorm(embedding_dim)
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, x):
        residual = x
        x = self.layer_norm1(x)
        x = self.attention(x) + residual
        x = self.dropout(x)

        residual = x
        x = self.layer_norm2(x)
        x = self.feed_forward(x) + residual
        x = self.dropout(x)

        return x

class TransformerModel(nn.Module):
    def __init__(self):
        super(TransformerModel, self).__init__()
        self.token_embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.position_embeddings = nn.Embedding(block_size, embedding_dim)
        self.transformer_blocks = nn.ModuleList([TransformerBlock(embedding_dim, num_heads) for _ in range(num_layers)])
        self.layer_norm = nn.LayerNorm(embedding_dim)
        self.output_layer = nn.Linear(embedding_dim, vocab_size)
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=0.02)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            module.weight.data.normal_(mean=0.0, std=0.02)

    def forward(self, input_ids, targets=None):
        batch_size, seq_len = input_ids.size()

        token_embeddings = self.token_embeddings(input_ids)
        position_embeddings = self.position_embeddings(torch.arange(seq_len, device=device))
        x = token_embeddings + position_embeddings

        for block in self.transformer_blocks:
            x = block(x)

        x = self.layer_norm(x)
        logits = self.output_layer(x)

        if targets is None:
            loss = None
        else:
            loss = F.cross_entropy(logits.view(-1, vocab_size), targets.view(-1))

In [7]:
class SimpleTokenizer:
    def __init__(self, stoi, itos):
        self.stoi = stoi
        self.itos = itos

    def encode(self, text):
        return torch.tensor([self.stoi.get(c, self.stoi[' ']) for c in text], dtype=torch.long).unsqueeze(0)

    def decode(self, tokens):
        return ''.join(self.itos[token] for token in tokens if token in self.itos)

# Example character set mappings, replace these with your actual mappings
with open("input.txt", "r") as f:
    text = f.read()
chars = sorted(list(set(text)))  # Define 'text' variable as your dataset text content
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}

tokenizer = SimpleTokenizer(stoi, itos)

def generate_text(prompt, model, max_new_tokens=100):
    model.eval()
    with torch.no_grad():
        tokens = tokenizer.encode(prompt).to(device)
        generated_tokens = model.generate(tokens, max_new_tokens=max_new_tokens)
        generated_text = tokenizer.decode(generated_tokens[0].tolist())
    return generated_text

# Example of generating text for different types of prompts
def run_text_generation_tests(model):
    test_prompts = {
        "Coherence Test": "Once upon a time, there was a brave knight named",
        "Contextual Relevance Test": "Romeo stood beneath Juliet's balcony and whispered,",
        "Stylistic Alignment Test": "To be or not to be, that is",
        "Complexity Test": "In the darkest depths of the enchanted forest, the",
        "Cultural Reference Test": "Hark! What light through yonder window"
    }

    for test_name, prompt in test_prompts.items():
        print(f"{test_name} - Generated text:", generate_text(prompt, model, max_new_tokens=100))

# Assuming 'model' is your trained GPTLanguageModel instance
run_text_generation_tests(model)

NameError: name 'model' is not defined